In [2]:
import pandas as pd
import json
df = pd.read_csv('nba_alltime_leaders.csv')
cols = ['PLAYER', 'GP', 'PTS', 'FT%', 'FTM', 'FTA', 'FGM','FGA','FG%','3PM','3PA','3P%','OREB','DREB','REB','AST','STL','BLK','TOV']
data = df[cols].sort_values("FT%", ascending=False).drop_duplicates(subset=['PLAYER'])
data = data.reset_index()[cols]
data_dict = data.to_dict(orient='records')
with open('players_ft.json', 'w') as json_file:
    json.dump(data_dict, json_file, indent=4)




In [34]:
result = []
for key, value in data.iterrows():
    d = {}
    #print(value['PLAYER'])
    #print(value['FT%'])
    d['Player'] = value['PLAYER']
    d['ft%'] = value['FT%']
    result.append(d)

In [35]:
result

[{'Player': 'Stephen Curry', 'ft%': 91.0},
 {'Player': 'Damian Lillard', 'ft%': 89.7},
 {'Player': 'Kyrie Irving', 'ft%': 88.6},
 {'Player': 'Kevin Durant', 'ft%': 88.4},
 {'Player': 'Desmond Bane', 'ft%': 87.9},
 {'Player': 'Khris Middleton', 'ft%': 87.9},
 {'Player': 'Jordan Poole', 'ft%': 87.8},
 {'Player': 'Danilo Gallinari', 'ft%': 87.6},
 {'Player': 'Malcolm Brogdon', 'ft%': 87.3},
 {'Player': 'Anfernee Simons', 'ft%': 87.3},
 {'Player': 'Trae Young', 'ft%': 87.3},
 {'Player': 'Isaiah Thomas', 'ft%': 87.2},
 {'Player': 'Devin Booker', 'ft%': 87.0},
 {'Player': 'Chris Paul', 'ft%': 87.0},
 {'Player': 'CJ McCollum', 'ft%': 86.9},
 {'Player': 'Kawhi Leonard', 'ft%': 86.2},
 {'Player': 'James Harden', 'ft%': 86.1},
 {'Player': 'Bojan Bogdanovic', 'ft%': 85.9},
 {'Player': 'Klay Thompson', 'ft%': 85.8},
 {'Player': 'Jeremy Lamb', 'ft%': 85.7},
 {'Player': 'Tyrese Haliburton', 'ft%': 85.6},
 {'Player': 'Austin Reaves', 'ft%': 85.5},
 {'Player': 'Cam Thomas', 'ft%': 85.5},
 {'Player': '

In [10]:
import json
# Replace 'your_file.json' with the path to your JSON file
with open('players_ft.json', 'r') as file:
    data = json.load(file)

d = {}
for p in data:
    d[p['PLAYER']] =  p['id']
with open('playerIndex.json', 'w') as file:
    json.dump(d, file, indent=4)


In [16]:
with open('playerIndex.json', 'r') as file:
    playerIndex = json.load(file)

with open('players_ft.json', 'r') as file:
    playerData = json.load(file)
for player in playerData:
    tempPlayer = player['PLAYER']
    if tempPlayer in playerIndex:
        player['index'] = playerIndex[tempPlayer]

In [20]:
with open('playerData.json', 'w') as json_file:
    json.dump(playerData, json_file, indent=4)

In [24]:
# Converts the JSON to SQLite database
import json
import sqlite3

# Load JSON data
with open('nbaPlayerData.json', 'r') as f:
    players_data = json.load(f)

# Connect to SQLite database (or create it)
conn = sqlite3.connect('nbaPlayers.db')
cursor = conn.cursor()

# Create the table with "index" quoted
cursor.execute('''
CREATE TABLE IF NOT EXISTS players (
    id INTEGER PRIMARY KEY,
    PLAYER TEXT,
    GP INTEGER,
    PTS INTEGER,
    FT_Percentage REAL,
    FTM INTEGER,
    FTA INTEGER,
    FGM INTEGER,
    FGA INTEGER,
    FG_Percentage REAL,
    ThreePM INTEGER,
    ThreePA INTEGER,
    ThreeP_Percentage REAL,
    OREB INTEGER,
    DREB INTEGER,
    REB INTEGER,
    AST INTEGER,
    STL INTEGER,
    BLK INTEGER,
    TOV INTEGER,
    PICIDX INTEGER
)
''')

# Insert data into the table
for player in players_data:
    cursor.execute('''
    INSERT INTO players (
        PLAYER, GP, PTS, FT_Percentage, FTM, FTA, FGM, FGA, FG_Percentage, 
        ThreePM, ThreePA, ThreeP_Percentage, OREB, DREB, REB, AST, STL, BLK, TOV, "index"
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        player['PLAYER'], player['GP'], player['PTS'], player['FT%'], player['FTM'], 
        player['FTA'], player['FGM'], player['FGA'], player['FG%'], player['3PM'], 
        player['3PA'], player['3P%'], player['OREB'], player['DREB'], player['REB'], 
        player['AST'], player['STL'], player['BLK'], player['TOV'], player['index']
    ))
    

# Commit changes and close the connection
conn.commit()
conn.close()

print("Data inserted successfully!")


Data inserted successfully!


In [22]:
with open('tempPlayerData.json', 'r') as f:
    players_data = json.load(f)
new_players = []
for player in players_data:
    if 'index' in player:
        new_players.append(player)

with open('nbaPlayerData.json', 'w') as file:
    json.dump(new_players, file, indent=4)

In [25]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('nbaPlayers.db')
cursor = conn.cursor()

# Query to fetch all data from the players table
cursor.execute('SELECT * FROM players')

# Fetch all rows
rows = cursor.fetchall()

# Print the column names
column_names = [description[0] for description in cursor.description]
print("Column Names:", column_names)

# Print each row
for row in rows:
    print(row)

# Close the connection
conn.close()


Column Names: ['id', 'PLAYER', 'GP', 'PTS', 'FT_Percentage', 'FTM', 'FTA', 'FGM', 'FGA', 'FG_Percentage', 'ThreePM', 'ThreePA', 'ThreeP_Percentage', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'index']
(1, 'Stephen Curry', 956, 23668, 91.0, 3753, 4125, 8084, 17098, 47.3, 3747, 8805, 42.6, 629, 3880, 4509, 6119, 1473, 235, 2987, 201939)
(2, 'Damian Lillard', 842, 21151, 89.7, 4900, 5460, 6822, 15576, 43.8, 2607, 7032, 37.1, 505, 3068, 3573, 5659, 804, 257, 2379, 203081)
(3, 'Reggie Miller', 144, 2972, 89.3, 770, 862, 941, 2095, 44.9, 320, 820, 39.0, 77, 339, 416, 357, 146, 34, 257, 397)
(4, 'Dirk Nowitzki', 145, 3663, 89.2, 1074, 1204, 1220, 2642, 46.2, 149, 408, 36.5, 212, 1234, 1446, 360, 149, 129, 328, 1717)
(5, 'Larry Bird', 164, 3897, 89.0, 901, 1012, 1458, 3090, 47.2, 80, 249, 32.1, 360, 1323, 1683, 1062, 296, 145, 506, 1449)
(6, 'Kyrie Irving', 729, 17199, 88.6, 2787, 3146, 6340, 13368, 47.4, 1732, 4404, 39.3, 578, 2355, 2933, 4134, 956, 314, 1831, 202681)
(7, 'Kevin Duran

In [ ]:
# Importing new data for leaderbaord
import pandas as pd

pd.read_html('https://www.basketball-reference.com/leagues/NBA_2020_totals.html#totals_stats')

[        Rk                 Player   Age Team  Pos     G    GS      MP     FG  \
 0      1.0           James Harden  30.0  HOU   SG  68.0  68.0  2483.0  672.0   
 1      2.0         Damian Lillard  29.0  POR   PG  66.0  66.0  2474.0  624.0   
 2      3.0           Devin Booker  23.0  PHO   SG  70.0  70.0  2512.0  627.0   
 3      4.0  Giannis Antetokounmpo  25.0  MIL   PF  63.0  63.0  1917.0  685.0   
 4      5.0             Trae Young  21.0  ATL   PG  60.0  60.0  2120.0  546.0   
 ..     ...                    ...   ...  ...  ...   ...   ...     ...    ...   
 647  648.0         William Howard  26.0  HOU   PF   2.0   0.0    13.0    0.0   
 648  649.0           Stanton Kidd  27.0  UTA   SG   4.0   0.0    15.0    0.0   
 649  650.0            J.P. Macura  24.0  CLE   SG   1.0   0.0     1.0    0.0   
 650  651.0            Isaiah Roby  21.0  OKC   PF   3.0   0.0    11.0    0.0   
 651    NaN         League Average   NaN  NaN  NaN   NaN   NaN     NaN    NaN   
 
         FGA  ...    DRB  

In [22]:
import pandas as pd
url_attempt = "https://www.basketball-reference.com/leaders/fta_career.html"
df_attempt = pd.read_html(url_attempt)[1]
url_made = "https://www.basketball-reference.com/leaders/ft_career.html"
df_made = pd.read_html(url_made)[1]
df = pd.merge(df_made, df_attempt, on='Player')
cols = ['Player', 'FT', 'FTA']
df = df[cols]
df['Percentage'] = df['FT'] / df['FTA']
df = df.sort_values(by='Percentage', ascending=False)
df = df.reset_index()
cols2 = ['Player', 'FT', 'FTA', 'Percentage']
df = df[cols2]
df["Player"] = df["Player"].str.replace(r"\*$", "", regex=True)

In [23]:
import json

with open('playerIndex.json', 'r') as file:
    playerIndexData = json.load(file)

import pandas as pd

dfPlayerIds = pd.DataFrame(
    list(playerIndexData.items()),
    columns=["Player", "Id"]
)
dfPlayerIds

,Player,Id
0,Stephen Curry,201939
1,Jared Mccain,1642272
2,Damian Lillard,203081
3,Reggie Miller,397
4,Dirk Nowitzki,1717
...,...,...
143,Kevin Garnett,708
144,Chris Bosh,2547
145,Jalen Rose,147
146,Joe Johnson,2207


In [24]:
final_df = pd.merge(df, dfPlayerIds, on='Player')
final_df

,Player,FT,FTA,Percentage,Id
0,Stephen Curry,4098,4494,0.911883,201939
1,Steve Nash,3060,3384,0.904255,959
2,Damian Lillard,5262,5853,0.899026,203081
3,Chauncey Billups,4496,5029,0.894015,1497
4,Ray Allen,4398,4920,0.893902,951
...,...,...,...,...,...
82,Rudy Gobert,2671,4176,0.639607,203497
83,Dwight Howard,5361,9455,0.567002,2730
84,Bill Russell,3148,5614,0.560741,78049
85,Shaquille O'Neal,5935,11252,0.527462,406


In [25]:
from nba_api.stats.endpoints import playercareerstats

# Nikola Jokić
career = playercareerstats.PlayerCareerStats(player_id='2544')

# pandas data frames (optional: pip install pandas)
df_regular_season = career.season_totals_regular_season.get_data_frame()

df_post_season = career.career_totals_post_season.get_data_frame()
# # json
json_version = career.get_json()

# # dictionary
dictionary = career.get_dict()

In [26]:
df_post_season

,PLAYER_ID,LEAGUE_ID,TEAM_ID,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,2544,00,0,292,292,12062,2971,5984,0.496491,480,...,0.741167,442,2186,2628,2095,493,284,1047,663,8289


In [27]:
df_regular_season

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,2544,2003-04,00,1610612739,CLE,19.0,79,79,3120,622,...,0.754,99,333,432,465,130,58,273,149,1654
1,2544,2004-05,00,1610612739,CLE,20.0,80,80,3388,795,...,0.750,111,477,588,577,177,52,262,146,2175
2,2544,2005-06,00,1610612739,CLE,21.0,79,79,3361,875,...,0.738,75,481,556,521,123,66,260,181,2478
3,2544,2006-07,00,1610612739,CLE,22.0,78,78,3190,772,...,0.698,83,443,526,470,125,55,250,171,2132
4,2544,2007-08,00,1610612739,CLE,23.0,75,74,3027,794,...,0.712,133,459,592,539,138,81,255,165,2250
5,2544,2008-09,00,1610612739,CLE,24.0,81,81,3054,789,...,0.780,106,507,613,587,137,93,241,139,2304
6,2544,2009-10,00,1610612739,CLE,25.0,76,76,2966,768,...,0.767,71,483,554,651,125,77,261,119,2258
7,2544,2010-11,00,1610612748,MIA,26.0,79,79,3063,758,...,0.759,80,510,590,554,124,50,284,163,2111
8,2544,2011-12,00,1610612748,MIA,27.0,62,62,2326,621,...,0.771,94,398,492,387,115,50,213,96,1683
9,2544,2012-13,00,1610612748,MIA,28.0,76,76,2877,765,...,0.753,97,513,610,551,129,67,226,110,2036


In [29]:
for id in final_df['Id']:
    print(id)

201939
959
203081
1497
951
397
1449
201142
1717
1626164
1629027
101108
1628983
2037
201935
208
202331
77142
202710
201942
101150
134
1626157
600015
977
893
201567
203954
1938
304
201144
203999
101114
203078
200768
2546
78497
1122
1718
201145
2240
2207
147
200752
2547
203076
708
950
947
76127
76681
201188
202322
101145
201566
77449
255
2210
2548
2216
78318
203944
2200
2225
1503
252
121
2544
200794
764
787
56
76003
1897
165
2738
937
1495
201933
203507
87
185
203497
2730
78049
406
76375


In [30]:
json_version

'{"resource": "playercareerstats", "parameters": {"PerMode": "Totals", "PlayerID": 2544, "LeagueID": "00"}, "resultSets": [{"name": "SeasonTotalsRegularSeason", "headers": ["PLAYER_ID", "SEASON_ID", "LEAGUE_ID", "TEAM_ID", "TEAM_ABBREVIATION", "PLAYER_AGE", "GP", "GS", "MIN", "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "OREB", "DREB", "REB", "AST", "STL", "BLK", "TOV", "PF", "PTS"], "rowSet": [[2544, "2003-04", "00", 1610612739, "CLE", 19.0, 79, 79, 3120, 622, 1492, 0.417, 63, 217, 0.29, 347, 460, 0.754, 99, 333, 432, 465, 130, 58, 273, 149, 1654], [2544, "2004-05", "00", 1610612739, "CLE", 20.0, 80, 80, 3388, 795, 1684, 0.472, 108, 308, 0.351, 477, 636, 0.75, 111, 477, 588, 577, 177, 52, 262, 146, 2175], [2544, "2005-06", "00", 1610612739, "CLE", 21.0, 79, 79, 3361, 875, 1823, 0.48, 127, 379, 0.335, 601, 814, 0.738, 75, 481, 556, 521, 123, 66, 260, 181, 2478], [2544, "2006-07", "00", 1610612739, "CLE", 22.0, 78, 78, 3190, 772, 1621, 0.476, 99, 310, 0.319

In [31]:
dictionary

{'resource': 'playercareerstats',
 'parameters': {'PerMode': 'Totals', 'PlayerID': 2544, 'LeagueID': '00'},
 'resultSets': [{'name': 'SeasonTotalsRegularSeason',
   'headers': ['PLAYER_ID',
    'SEASON_ID',
    'LEAGUE_ID',
    'TEAM_ID',
    'TEAM_ABBREVIATION',
    'PLAYER_AGE',
    'GP',
    'GS',
    'MIN',
    'FGM',
    'FGA',
    'FG_PCT',
    'FG3M',
    'FG3A',
    'FG3_PCT',
    'FTM',
    'FTA',
    'FT_PCT',
    'OREB',
    'DREB',
    'REB',
    'AST',
    'STL',
    'BLK',
    'TOV',
    'PF',
    'PTS'],
   'rowSet': [[2544,
     '2003-04',
     '00',
     1610612739,
     'CLE',
     19.0,
     79,
     79,
     3120,
     622,
     1492,
     0.417,
     63,
     217,
     0.29,
     347,
     460,
     0.754,
     99,
     333,
     432,
     465,
     130,
     58,
     273,
     149,
     1654],
    [2544,
     '2004-05',
     '00',
     1610612739,
     'CLE',
     20.0,
     80,
     80,
     3388,
     795,
     1684,
     0.472,
     108,
     308,
     0.351,
 